In [2]:
import numpy as np
import pandas as pd
import random
import time

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torchtext import data
import torch.optim as optim
import transformers
from transformers import BertTokenizer, BertModel

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [3]:
SEED = 2202

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [4]:
df = pd.read_csv('path_to_data.csv')
df.head(10)

,comment,amount
0,Спасибо большое за оперативный заказ! Доставил...,5
1,Пожарская котлета вкуснющая! Спасибо))),5
2,"Спасибо, спасибо, как всегда быстро и вкусно 👍👌",5
3,"На фото блинов, их там 5 У меня 3",1
4,яйцо пашот пришло вкрутую,-20
5,"Привет. Вкусно, но без украшений.",-20
6,Здравствуйте! Вчера заказывала салат с авокадо...,1
7,Упаковка не то что крутая - кажется она сохран...,5
8,"""Спасибо огромное! Быстро доставили""",5
9,"""Я случайно оплатил заказ дважды, отменил один...",-20


In [5]:
df_upd = df.copy()

df_upd['amount'].replace(1,0, inplace=True)
df_upd['amount'].replace(5,0, inplace=True)
df_upd['amount'].replace(-20,1, inplace=True)

df_upd['amount'].value_counts()

1    525
0    475
Name: amount, dtype: int64

In [6]:
train, test = train_test_split(df_upd, test_size=0.1,
                                             random_state=SEED, stratify=df_upd['amount'])

In [7]:
s_train = train.to_csv('s_train.csv', index=False)
s_test = test.to_csv('s_test.csv', index=False)

In [8]:
tokenizer = BertTokenizer(vocab_file='../input/deeppavlov-ru-bert-model-deeppavlov/vocab_rubert.txt')
len(tokenizer.vocab)

119547

In [9]:
tokens = tokenizer.tokenize(df.comment[0])
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[107685, 13842, 1758, 14992, 1468, 852, 12300, 106, 32803, 1758, 6762, 22337, 128, 4752, 36728, 88193, 842, 851, 28884, 2784, 106, 625, 74478, 7805, 68221, 33241, 852, 122, 122, 122, 122, 7805, 25567, 1468, 852, 21746, 128, 13231, 35676, 28636, 14825, 106, 122, 122]


In [10]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [11]:
MAX_INPUT = 512

def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:MAX_INPUT-2]
    return tokens

In [12]:
TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

fields = [('comment', TEXT), ('amount', LABEL)]

In [13]:
train_data = data.TabularDataset(
                            's_train.csv',
                            format = 'csv', 
                            skip_header=True,
                            fields = fields
)

test_data = data.TabularDataset(
                            's_test.csv',
                            format = 'csv', 
                            skip_header=True,
                            fields = fields
)

In [16]:
test_data[0].amount

'1'

In [17]:
print(test_data[0].amount)
print(test_data[0].comment)
print(len(test_data))

1
[34005, 845, 52702, 842, 13526, 3618, 33242, 116155, 1523]
100


In [18]:
print(train_data[0].amount)
print(train_data[0].comment)
print(len(train_data))

0
[3240, 880, 852, 2226, 74176, 901, 3446, 4768, 106, 100]
900


In [20]:
LABEL.build_vocab(train_data)
print(LABEL.vocab.stoi)

defaultdict(None, {'1': 0, '0': 1})


In [21]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size = (BATCH_SIZE), 
    sort=False,
    device = device)

In [22]:
config = transformers.BertConfig.from_json_file(
    '../input/bert-multi/bert_config.json')

bert = BertModel.from_pretrained(
    '../input/deeppavlov-ru-bert-model-deeppavlov/pytorch_model.bin', config=config)

In [23]:
class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()    
        self.bert = bert        
        embedding_dim = bert.config.to_dict()['hidden_size']     
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)     
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        with torch.no_grad():
            embedded = self.bert(text)[0]       
        _, hidden = self.rnn(embedded)

        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])

        output = self.out(hidden)

        return output

In [24]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [25]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 180,612,609 trainable parameters


In [26]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [27]:
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,759,169 trainable parameters


In [28]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

In [29]:
model = model.to(device)
criterion = criterion.to(device)

In [30]:
def binary_accuracy(preds, y):

    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() 
    acc = correct.sum() / len(correct)
    return acc

In [31]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()      
        predictions = model(batch.comment).squeeze(1)
        
        loss = criterion(predictions, batch.amount)
        acc = binary_accuracy(predictions, batch.amount)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [32]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.comment).squeeze(1)
            
            loss = criterion(predictions, batch.amount)
            acc = binary_accuracy(predictions, batch.amount)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [33]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [45]:
N_EPOCHS = 5

best_test_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    test_loss, test_acc = evaluate(model, test_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save(model.state_dict(), 'classification.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {test_loss:.3f} |  Val. Acc: {test_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 4s
	Train Loss: 0.373 | Train Acc: 83.65%
	 Val. Loss: 0.408 |  Val. Acc: 83.51%
Epoch: 02 | Epoch Time: 0m 4s
	Train Loss: 0.349 | Train Acc: 83.96%
	 Val. Loss: 0.406 |  Val. Acc: 84.11%
Epoch: 03 | Epoch Time: 0m 4s
	Train Loss: 0.326 | Train Acc: 86.46%
	 Val. Loss: 0.423 |  Val. Acc: 86.28%
Epoch: 04 | Epoch Time: 0m 4s
	Train Loss: 0.347 | Train Acc: 85.62%
	 Val. Loss: 0.366 |  Val. Acc: 86.46%
Epoch: 05 | Epoch Time: 0m 4s
	Train Loss: 0.425 | Train Acc: 79.17%
	 Val. Loss: 0.408 |  Val. Acc: 84.11%


In [35]:
def predict_example(sentence):
    model.eval()
    tokens = tokenizer.tokenize(sentence)
    tokens = tokens[:MAX_INPUT-2]
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [46]:
predict_example('у вас очень вкусный суп')

0.8517205119132996

In [47]:
predict_example('курьер опоздал')

0.1030353382229805

In [50]:
predict_example('вкусно, буду заказывать еще раз')

0.8868089914321899

In [51]:
predict_example('падал прошлогодний снег')

0.8771221041679382

In [52]:
predict_example('суп старый, паста твердая, пашот развалился')

0.30844855308532715